# Ingeniería de Características y Selección Basada en Correlación

En todo proceso de modelado supervisado, la calidad de las variables de entrada puede llegar a ser tan determinante como el propio algoritmo. En este notebook proponemos una estrategia estadística para segmentar nuestro espacio de características en dos grupos: uno con las **columnas más fuertemente correlacionadas** y otro con aquellas de **correlación moderada** con respecto a la variable objetivo. De este modo, podremos entrenar modelos con distintos niveles de información y comparar directamente su desempeño.

Primero, ampliamos nuestro set original de datos generando nuevas variables —por ejemplo, interacciones o polinomios— que potencialmente capten patrones no lineales. A continuación, medimos la relevancia de cada característica mediante el coeficiente de correlación de Pearson:

$$
r_i = \frac{\mathrm{cov}(X_i, Y)}{\sigma_{X_i}\,\sigma_{Y}}
$$

y cuantificamos la redundancia usando el **Factor de Inflación de la Varianza (VIF)**:

$$
\mathrm{VIF}_i = \frac{1}{1 - R_i^2}
$$

donde $R_i^2$ es el coeficiente de determinación al ajustar $X_i$ como variable dependiente frente al resto de columnas del dataset.

Para decidir qué columnas pasan a cada grupo, estudiamos la **distribución empírica** de los valores $|r_i|$. Definimos dos umbrales basados en percentiles:

- El **66.6 %** señala el corte para nuestra **Tabla “Relevante”** (correlaciones más altas).  
- El **33.3 %** marca el límite mínimo para la **Tabla “Semi-Relevante”** (correlaciones moderadas).

El resto de las variables, con $|r_i|$ por debajo del percentil 33.3, queda fuera del conjunto de entrenamiento por aportar muy poca señal.

En la siguiente sección presentamos una visualización con una curva normal de referencia:

![Distribución de correlaciones con la variable objetivo](Graphics\Ej_filtrado_correlacion.png)

Las áreas sombreadas a la derecha de cada umbral permiten apreciar de un solo vistazo la proporción de columnas seleccionadas.

Finalmente, crearemos dos DataFrames —uno con las columnas del grupo “Relevante” y otro con las del grupo “Semi-Relevante”— y aplicaremos la **ecuación de pronóstico** proporcionada por el concurso para predecir el número de unidades vendidas en los 16 días siguientes. Evaluaremos cada modelo comparando métricas de error de regresión como **MAE** (Error Absoluto Medio) y **RMSE** (Raíz del Error Cuadrático Medio) para determinar si el uso de un conjunto de variables más amplio (Semi-Relevante) mejora o empeora las predicciones frente al conjunto más restringido (Relevante). Esta metodología aporta rigor estadístico y flexibilidad para explorar el trade-off entre complejidad del modelo y precisión de la predicción.


Claro, aquí tienes una versión mejorada, más clara, coherente y profesional del texto, junto con la fórmula de variación porcentual diaria y un ejemplo más estructurado del análisis de desfase:

---

Vamos a utilizar como plantilla base la tabla `train`, a partir de la cual crearemos dos nuevas copias denominadas `Test_50_insight` y `Test_100_insight`.

Planeo trabajar con el DataFrame `oil`, del cual extraeremos las columnas `date` y `dcoilwtico` (precio del petróleo WTI). Con esta información, calcularemos la **variación porcentual diaria del precio del petróleo** y la almacenaremos en una nueva columna llamada `oil_price`.

Esta columna será posteriormente combinada con la tabla `train`, alineando los valores por la columna `date`, para así conocer cómo varió el precio del petróleo cada día específico.

### Cálculo de la variación porcentual diaria

La fórmula para calcular la variación porcentual diaria es:

$$
\text{variación}_{t} = \frac{P_{t} - P_{t-1}}{P_{t-1}} \times 100
$$

Donde:

* $P_t$ es el precio del petróleo en el día actual
* $P_{t-1}$ es el precio del día anterior

### Justificación del desfase temporal

Inicialmente se pensó en **adelantar un día** la variable `oil_price`, ya que el precio del petróleo actual podría influir en las decisiones del mercado **al día siguiente**, no en el mismo día en que se registra el precio. Sin embargo, para tomar una decisión informada, se propone realizar un análisis de **correlación** entre la variable `oil_price` y otras variables objetivo, considerando tres escenarios distintos:

1. **Desfase -1 (precio del día anterior):**
   Se utiliza el precio del petróleo del día anterior para predecir el comportamiento del día actual.
2. **Sin desfase (precio actual):**
   Se utiliza el precio del petróleo del mismo día.
3. **Desfase +1 (precio del día siguiente):**
   Se usa el precio del día siguiente, bajo la hipótesis de que el precio de hoy refleja la reacción al comportamiento del día anterior.

### Ejemplo ilustrativo de desfases

Aquí tienes un ejemplo más claro de cómo funciona el desfase en los precios del petróleo. Supongamos que los precios diarios son los siguientes:

| Fecha      | Precio WTI |
| ---------- | ---------- |
| 17 de mayo | 1.01       |
| 18 de mayo | 1.05       |
| 19 de mayo | 1.02       |

Y que nuestra plantilla `train` contiene las mismas fechas como índice para unir:

| Fecha      | … otras columnas … |
| ---------- | ------------------ |
| 17 de mayo | …                  |
| 18 de mayo | …                  |
| 19 de mayo | …                  |

Ahora veamos los tres escenarios:

---

### 1. Desfase -1 (precio del día anterior)

Aquí asignamos al 18 de mayo el precio del 17 de mayo, y al 19 de mayo el del 18 de mayo:

| Fecha (train) | oil\_price (t-1) |
| ------------- | ---------------- |
| 17 de mayo    | —                |
| 18 de mayo    | 1.01             |
| 19 de mayo    | 1.05             |

* **Interpretación**: Para el 18 usas el precio del 17, asumiendo que el mercado reacciona con un día de retraso.

---

### 2. Sin desfase (precio del mismo día)

Asignamos a cada fecha su propio precio:

| Fecha (train) | oil\_price (t) |
| ------------- | -------------- |
| 17 de mayo    | 1.01           |
| 18 de mayo    | 1.05           |
| 19 de mayo    | 1.02           |

* **Interpretación**: El modelo ve el precio del petróleo del mismo día.

---

### 3. Desfase +1 (precio del día siguiente)

Aquí adelantamos un día: al 17 de mayo le ponemos el precio del 18, y al 18 el del 19:

| Fecha (train) | oil\_price (t+1) |
| ------------- | ---------------- |
| 17 de mayo    | 1.05             |
| 18 de mayo    | 1.02             |
| 19 de mayo    | —                |

* **Interpretación**: El modelo “ve” el precio que estará disponible mañana, ideal si crees que las señales del petróleo ya anticipan movimientos un día antes.

---

Con estos tres conjuntos de datos podrás calcular el coeficiente de correlación de cada versión de `oil_price` con tu variable objetivo y decidir cuál desfase aporta más valor predictivo.


## `HOLIDAYS_Events`

Este es otro *dataframe* que contiene columnas con información útil que podemos aprovechar.  
La idea es crear dos nuevas columnas en nuestro conjunto de datos `train` (y también `test`):

1. **`es_festivo`**: una columna booleana que indicará si la fecha corresponde a un día festivo (`True` o `False`).
2. **`tipo_festivo`**: una columna categórica que especificará el tipo de día festivo. Para facilitar su uso en modelos de IA (que no aceptan variables de tipo `string` directamente), se codificará de forma numérica:

   - `0`: festivo local  
   - `1`: festivo regional  
   - `2`: festivo nacional

> ⚠️ **Nota importante:**  
Para clasificar correctamente si un día es festivo y su tipo, se debe tener en cuenta la información contenida en `stores.csv`.  
Cada fila de los dataframes `train` y `test` contiene una columna llamada `store_nbr`, que representa el número de tienda.  
Mediante este número, se puede localizar la tienda correspondiente en `stores.csv`, que contiene datos clave como la ciudad, estado o región de cada tienda.  
Posteriormente, al cruzar esa ubicación con la fecha correspondiente en el dataframe `HOLIDAYS_Events`, es posible determinar:

- Si ese día es festivo o no para esa tienda (`es_festivo`)
- Y en caso afirmativo, el tipo de festivo (`tipo_festivo`: local, regional o nacional)

Estas nuevas columnas nos permitirán enriquecer el análisis y posiblemente mejorar el rendimiento del modelo.


Del DataFrame `items` extraeremos las columnas `family`, `class` y `perishable` con el objetivo de convertirlas a variables numéricas. En particular, la columna `perishable` se transformará en una variable binaria, donde:

- `1` indicará que el producto es perecedero,
- `0` que no lo es.

Por otro lado, del DataFrame `transactions` extraeremos la columna `transactions`, ya que la utilizaremos como variable explicativa en el análisis.


## Estructura Final del DataFrame Integrado

A continuación se muestra un ejemplo de cómo quedaría la arquitectura final del DataFrame después de integrar y transformar las distintas fuentes de datos (`sales`, `items`, `transactions`, `holidays_events`, `oil`), dejando todas las columnas en formato numérico:

| id  | date       | store_nbr | item_nbr | unit_sales | onpromotion | is_festive | type_festive | type_local_festive | Price_oil_pct | family_items | class_items | perishable | transactions |
|-----|------------|-----------|----------|------------|-------------|------------|--------------|---------------------|----------------|---------------|-------------|------------|--------------|
| 1   | 2017-08-15 | 1         | 103520   | 3.0        | 1           | 1          | 2            | 1                   | 0.012          | 4             | 1013        | 1          | 1345         |
| 2   | 2017-08-15 | 1         | 105574   | 0.0        | 0           | 1          | 2            | 1                   | 0.012          | 2             | 2020        | 1          | 1345         |
| 3   | 2017-08-16 | 2         | 103520   | 5.0        | 1           | 0          | 0            | 0                   | -0.006         | 4             | 1013        | 1          | 1120         |
| 4   | 2017-08-16 | 2         | 209211   | 8.0        | 0           | 0          | 0            | 0                   | -0.006         | 3             | 3001        | 1          | 1120         |

### Descripción de columnas

- **`store_nbr`**: Identificador numérico de la tienda.
- **`item_nbr`**: Identificador numérico del producto.
- **`unit_sales`**: Unidades vendidas del producto en esa fecha y tienda.
- **`onpromotion`**: Indicador binario (`1` si el producto estaba en promoción, `0` si no).
- **`is_festive`**: Indicador binario (`1` si la fecha es festiva, `0` si no).
- **`type_festive`**: Tipo de festividad codificado numéricamente (ej. `0`: Ninguna, `1`: Evento, `2`: Holiday).
- **`type_local_festive`**: Indicador binario (`1` si la festividad es local, `0` si no).
- **`Price_oil_pct`**: Variación porcentual del precio del petróleo respecto al día anterior.
- **`family_items`**: Categoría general del producto codificada numéricamente (Label Encoding).
- **`class_items`**: Subcategoría o clase numérica del producto.
- **`perishable`**: Indicador binario (`1` si el producto es perecedero, `0` si no).
- **`transactions`**: Número total de transacciones realizadas en la tienda ese día.

Este DataFrame está completamente listo para ser utilizado en modelos de predicción o análisis multivariado, cumpliendo con el requisito de tener únicamente variables numéricas.


In [9]:
import pandas as pd
from tqdm import tqdm

# 1) Cargo y preparo el df pequeño de precios de crudo
oil_df = pd.read_csv(
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\processed\clear_oil_raw_Dukascopy.csv",
    parse_dates=["date"]
)
oil_df = oil_df.sort_values("date")
oil_df["pct_change"] = oil_df["dcoilwtico"].pct_change() * 100
oil_df["pct_change"] = oil_df["pct_change"].fillna(0)
oil_df = oil_df[["date", "pct_change"]].rename(columns={"pct_change": "Price_oil_pct"})

# 1b) Cargo el dataframe de transacciones
transactions_df = pd.read_csv(
    r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\raw\transactions.csv",
    parse_dates=["date"]
)
# Asegúrate de que store_nbr tenga el mismo tipo (int o float) en ambos dfs
transactions_df["store_nbr"] = transactions_df["store_nbr"].astype(oil_df.dtypes.get("store_nbr", "int64"))

# 2) Configuración de paths y tamaño de chunk
input_path   = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\processed\clear_train.csv"
output_path  = r"D:\Portafolio oficial\Retail Sales Trend Analysis\data\data\processed\train_with_oil_and_tx.csv"
chunksize    = 300_000   # ajústalo a tu memoria disponible

# 3) Itero con tqdm para ver el progreso
first_chunk = True
for chunk in tqdm(
    pd.read_csv(input_path, parse_dates=["date"], chunksize=chunksize),
    desc="Procesando chunks",
    unit="chunk"
):
    # Si store_nbr viene como float, conviértelo a int para emparejar con transactions_df
    chunk["store_nbr"] = chunk["store_nbr"].astype(transactions_df["store_nbr"].dtype)
    
    # Merge con precio del crudo
    merged = chunk.merge(oil_df, on="date", how="left")
    # Merge con transacciones por fecha y tienda
    merged = merged.merge(
        transactions_df,
        on=["date", "store_nbr"],
        how="left"
    )
    
    # Si hay fechas/tiendas sin transacciones, puedes rellenar con 0:
    merged["transactions"] = merged["transactions"].fillna(0).astype(int)
    
    # Escribo al CSV final, añadiendo cabecera solo la primera vez
    merged.to_csv(
        output_path,
        mode="w" if first_chunk else "a",
        header=first_chunk,
        index=False
    )
    first_chunk = False


Procesando chunks: 419chunk [14:30,  2.08s/chunk]
